In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta

# jax
import jax.numpy as jnp
import time

import sys
import os
import importlib
import pickle
import itertools

path_ = os.path.abspath(os.path.join('..', '00_utils'))
if path_ not in sys.path:
    sys.path.append(path_)

path_ = os.path.abspath(os.path.join('..', '00_models'))
if path_ not in sys.path:
    sys.path.append(path_)

path_ = os.path.abspath(os.path.join('..', '00_utils_training'))
if path_ not in sys.path:
    sys.path.append(path_)

import run_train_pyomo_rl

import logging
logging.basicConfig(level=logging.ERROR, filename='error_log.txt')

In [8]:
START_DATE = '2015-01-05'
start_date = datetime.strptime(START_DATE, '%Y-%m-%d')
date_sequences = [start_date + timedelta(days=i*2) for i in range(20)]
date_sequences_str = [date.strftime('%Y-%m-%d') for date in date_sequences]
print(date_sequences_str)

times_elapsed = []

['2015-01-05', '2015-01-07', '2015-01-09', '2015-01-11', '2015-01-13', '2015-01-15', '2015-01-17', '2015-01-19', '2015-01-21', '2015-01-23', '2015-01-25', '2015-01-27', '2015-01-29', '2015-01-31', '2015-02-02', '2015-02-04', '2015-02-06', '2015-02-08', '2015-02-10', '2015-02-12']


In [9]:
def initialize_metrics():
    # Initialize lists to store metrics for each iteration
    metrics = {
        'times_elapsed': [],
        'mse_odeint': [],
        'mse_coll_ode': [],
        'mse_odeint_test': [],
        'mse_coll_ode_test': []
    }
    return metrics

def collect_metrics(metrics, experiment_results):
    # Extract metrics from the results dictionary and append to the respective lists
    metrics['times_elapsed'].append(experiment_results['times_elapsed'])
    metrics['mse_odeint'].append(experiment_results['mse_odeint'])
    metrics['mse_coll_ode'].append(experiment_results['mse_coll_ode'])
    metrics['mse_odeint_test'].append(experiment_results['mse_odeint_test'])
    metrics['mse_coll_ode_test'].append(experiment_results['mse_coll_ode_test'])

def compute_averages(metrics):
    # Compute average of each metric and return as a new dictionary
    averages = {key: sum(values) / len(values) for key, values in metrics.items()}
    return averages

* [X] Regularization Strength with Fixed Tolerance
* [X] 'tol' with no regularization
* [X] 'tol - 'reg' cross-validation

In [4]:
param_combinations_input_features = [{'prev_hour':True, 'prev_week':True, 'prev_year':True, 'm':1,'ls': [8, 32, 1]}, 
                        {'prev_hour':True, 'prev_week':True, 'prev_year':False, 'm':1, 'ls': [7, 32, 1]}, 
                        {'prev_hour':True, 'prev_week':False, 'prev_year':True, 'm':1, 'ls': [7, 32, 1]}, 
                        {'prev_hour':True, 'prev_week':False, 'prev_year':False, 'm':1, 'ls': [6, 32, 1]}, 
                        {'prev_hour':False, 'prev_week':True, 'prev_year':True, 'm':1, 'ls': [6, 32, 1]}, 
                        {'prev_hour':False, 'prev_week':False, 'prev_year':True, 'm':1, 'ls': [6, 32, 1]}, 
                        {'prev_hour':False, 'prev_week':True, 'prev_year':False, 'm':1, 'ls': [6, 32, 1]}, 
                        {'prev_hour':False, 'prev_week':False, 'prev_year':False, 'm':1, 'ls': [5, 32, 1]}, 
                        {'prev_hour':True, 'prev_week':True, 'prev_year':True, 'm':0,'ls': [7, 32, 1]}, 
                        {'prev_hour':True, 'prev_week':True, 'prev_year':False, 'm':0, 'ls': [6, 32, 1]}, 
                        {'prev_hour':True, 'prev_week':False, 'prev_year':True, 'm':0, 'ls': [6, 32, 1]}, 
                        {'prev_hour':True, 'prev_week':False, 'prev_year':False, 'm':0, 'ls': [5, 32, 1]}, 
                        {'prev_hour':False, 'prev_week':True, 'prev_year':True, 'm':0, 'ls': [5, 32, 1]}, 
                        {'prev_hour':False, 'prev_week':False, 'prev_year':True, 'm':0, 'ls': [5, 32, 1]}, 
                        {'prev_hour':False, 'prev_week':True, 'prev_year':False, 'm':0, 'ls': [5, 32, 1]}, 
                        {'prev_hour':False, 'prev_week':False, 'prev_year':False, 'm':0, 'ls': [4, 32, 1]}, 
                        ]

In [10]:
importlib.reload(run_train_pyomo_rl)
# aim: loop accross different penalties and see how the results change
# average results over different starting dates

LS = [7, 32, 1]
PENALTY = 1e-5
RESULTS_FULL= {}
RESULTS_AVG = {}

# ---------------------------- UPDATE OPTIMIZATION TYPE ----------------------------------
optimization_aim = 'collocation_method'

params_data = {'file_path': '../00_data/df_train.csv', 'start_date': START_DATE, 
            'n_points': 400, 'split': 200, 'n_days': 1, 'm': 1, 'prev_hour': True, 'prev_week': True, 'prev_year': False,
            'encoding': {'settlement_date': 't', 'temperature': 'var1', 'hour': 'var2', 'nd': 'y',
                         'spacing':'gauss_radau'},}

params_solver = {
    "tol" : 1e-6, 
    #"dual_inf_tol": 1e-4, 
    #"compl_inf_tol": 1e-3, 
    #"constr_viol_tol": 1e-5, 
    "acceptable_constr_viol_tol": 1e-15,
    "acceptable_dual_inf_tol": 1e-15,
    "acceptable_compl_inf_tol": 1e-15,
    "halt_on_ampl_error" : 'yes', "print_level": 1, "max_iter": 500,
    'warm_start_init_point': 'yes'}

params_ode = {
    "print_level": 1
}

if optimization_aim == 'regularization':
    param_combinations = [0, 1e-7, 1e-5, 1e-3, 0.01, 0.1]
elif optimization_aim == 'tolerance':
    param_combinations = [1, 1e-1, 1e-2, 1e-4, 1e-6, 1e-8]
elif optimization_aim == 'tolerance_mix':
    tol = [1e-4, 1e-6, 1e-8]
    tol_inf_and_viol = [1e-2, 1e-4, 1e-6]
    tol_dual_inf = [10, 1, 1e-1]
    param_combinations = list(itertools.product(tol, tol_inf_and_viol, tol_inf_and_viol, tol_dual_inf))
elif optimization_aim == 'reg_tol':
    penalty_values = [0, 1e-7, 1e-5, 1e-3, 0.01, 0.1]
    tol_list = [1, 1e-1, 1e-2, 1e-4, 1e-6, 1e-8]
    param_combinations = list(itertools.product(penalty_values, tol_list))
elif optimization_aim == 'input_features':
    param_combinations = param_combinations_input_features
elif optimization_aim == 'collocation_method':
    param_combinations = ["chebyshev", "gauss_legendre", "gauss_radau", "gauss_lobatto"]
else:
    raise ValueError("optimization_aim not recognized")

with open('results.txt', 'w'):
    pass
file = open('results.txt', 'a')

iter = 1

for param_comb in param_combinations:
    print(['*' for i in range(15)])
    print('Have you updated optimization_aim?')
    print(['*' for i in range(15)])
    
    # ------------------------------------------ UPDATE PARAMETERS ------------------------------------------
    if optimization_aim == 'regularization':
        PENALTY = param_comb
    elif optimization_aim == 'tolerance':
        params_solver['tol'] = param_comb
    elif optimization_aim == 'tolerance_mix':
        params_solver['tol'] = param_comb[0]
        params_solver['constr_viol_tol'] = param_comb[1]
        params_solver['dual_inf_tol'] = param_comb[2]
        params_solver['compl_inf_tol'] = param_comb[3]
    elif optimization_aim == 'reg_tol':
        PENALTY = param_comb[0]
        params_solver['tol'] = param_comb[1]
        params_solver['constr_viol_tol'] = param_comb[1]
    elif optimization_aim == 'input_features':   
        params_data['prev_hour'] = param_comb['prev_hour']
        params_data['prev_week'] = param_comb['prev_week']
        params_data['prev_year'] = param_comb['prev_year']
        params_data['m'] = param_comb['m']
        LS = param_comb['ls']
    elif optimization_aim == 'collocation_method':
        params_data['spacing'] = param_comb
        
        params_solver['constr_viol_tol'] = 1e-5
    
    metrics = initialize_metrics()
    
    for i, date in enumerate(date_sequences):
        print(f"param_comb: {param_comb}, date: {date}")
        
        # ------------------------------------------- UPDATE DATE -------------------------------------------
        params_data['start_date'] = date
        Trainer = run_train_pyomo_rl.Trainer
        params_model = {'layer_sizes': LS, 'penalty': PENALTY}
        trainer = Trainer(params_data, params_model, params_solver, params_ode)
        
        if optimization_aim == 'input_features':
            param_comb = str(param_comb)
        
        # ---------------------------------------------- TRAIN ---------------------------------------------
        try:
            experiment_results = trainer.train()
        except Exception as e:
            logging.error("Failed to complete training: {}".format(e))
            print(f"{e}")
            continue
        
        try:
            RESULTS_FULL[(param_comb, date)] = experiment_results
        except Exception as e:
            logging.error("Failed to extract results: {}".format(e))
            print(f"{e}")
            continue
            
        collect_metrics(metrics, experiment_results)
        print(f"param_comb: {param_comb}, date: {date}, results: {experiment_results}")
        
        file.write(f"param_comb: {param_comb}, date: {date}, results: {experiment_results}\n")
        file.flush() 
        
        iter += 1
        print (f"i: {iter}/{len(param_combinations)*len(date_sequences)}")

    try:
        RESULTS_AVG[param_comb] = compute_averages(metrics)
    except Exception as e:
        logging.error("Failed to save results: {}".format(e))
        print(f"{e}")

['*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*']
Have you updated optimization_aim?
['*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*']
param_comb: chebyshev, date: 2015-01-05 00:00:00
current_16_08
y
Ipopt 3.14.16: tol=1e-06
acceptable_constr_viol_tol=1e-15
acceptable_dual_inf_tol=1e-15
acceptable_compl_inf_tol=1e-15
halt_on_ampl_error=yes
print_level=1
max_iter=500
warm_start_init_point=yes
constr_viol_tol=1e-05


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Ipopt 3.14.16: print_level=1


******************************************************************************
This program contains Ipopt, a li

In [11]:
reload = True
if reload:
    formatted_time = time.strftime('%Y-%m-%d_%H-%M-%S')
    filename = f'results/{formatted_time}_spacing_type_full.pkl'
    with open(filename, 'wb') as file:
        pickle.dump(RESULTS_FULL, file)

In [51]:
RESULTS_FULL

{((0, 1),
  datetime.datetime(2015, 1, 5, 0, 0)): {'result': {'solver_time': 1.6005430221557617,
   'termination_condition': <TerminationCondition.optimal: 'optimal'>,
   'message': 'Ipopt 3.14.16\\x3a Optimal Solution Found'}, 'times_elapsed': 1.6005430221557617, 'mse_odeint': 0.5830821697950568, 'mse_coll_ode': 0.6147958872408735, 'mse_odeint_test': 1.479710590973089, 'mse_coll_ode_test': 1.6462049084694923},
 ((0, 1),
  datetime.datetime(2015, 1, 7, 0, 0)): {'result': {'solver_time': 1.2286438941955566,
   'termination_condition': <TerminationCondition.optimal: 'optimal'>,
   'message': 'Ipopt 3.14.16\\x3a Optimal Solution Found'}, 'times_elapsed': 1.2286438941955566, 'mse_odeint': 0.5657806971691728, 'mse_coll_ode': 0.7864454017035467, 'mse_odeint_test': 27.59196760896826, 'mse_coll_ode_test': 14.778510588090544},
 ((0, 1),
  datetime.datetime(2015, 1, 9, 0, 0)): {'result': {'solver_time': 1.5340018272399902,
   'termination_condition': <TerminationCondition.optimal: 'optimal'>,
  

In [26]:
RESULTS_AVG

{"{'prev_hour': True, 'prev_week': True, 'prev_year': True, 'm': 1, 'ls': [8, 32, 1]}": {'times_elapsed': 3.839753437042236,
  'mse_odeint': 1133.1437665698681,
  'mse_coll_ode': 0.3306330660892298,
  'mse_odeint_test': 1376.727114586666,
  'mse_coll_ode_test': 0.7285234841892397},
 "{'prev_hour': True, 'prev_week': True, 'prev_year': False, 'm': 1, 'ls': [7, 32, 1]}": {'times_elapsed': 4.147067022323609,
  'mse_odeint': 1104.385847924938,
  'mse_coll_ode': 0.7214805046384071,
  'mse_odeint_test': 1360.3313835812983,
  'mse_coll_ode_test': 0.6654537762226783},
 "{'prev_hour': True, 'prev_week': False, 'prev_year': True, 'm': 1, 'ls': [7, 32, 1]}": {'times_elapsed': 4.70120940208435,
  'mse_odeint': 1211.4729224901075,
  'mse_coll_ode': 0.5567925969264895,
  'mse_odeint_test': 1424.3330150764918,
  'mse_coll_ode_test': 0.6968927070303951},
 "{'prev_hour': True, 'prev_week': False, 'prev_year': False, 'm': 1, 'ls': [6, 32, 1]}": {'times_elapsed': 3.335993766784668,
  'mse_odeint': 1155.4